## SQLAlchemy assincrono
***

Do sincrono para o assincrono o que mais muda são as conexões, o resto é só colocar async e await nos locais corretos e tudo é basicamente a mesma coisa.

In [1]:
from sqlalchemy.ext.asyncio import create_async_engine, async_sessionmaker, AsyncSession
from sqlalchemy import URL

url = URL.create(
    drivername="postgresql+asyncpg",
    username='notebook',
    password='notebook',
    host='postgres',
    database='notebook',
    port=5432
)
engine = create_async_engine(url, pool_size=10, max_overflow=20, pool_recycle=3600)
Session = async_sessionmaker(bind=engine)

In [2]:
from sqlalchemy import INTEGER
from sqlalchemy.orm import DeclarativeBase


class Base(DeclarativeBase):
    pass

In [3]:
import datetime
from typing import Optional
from sqlalchemy.dialects.postgresql import TIMESTAMP
from sqlalchemy.sql.functions import func
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import ForeignKey, BIGINT, VARCHAR, String, DECIMAL

class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    updated_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


class User(Base, TimestampMixin):
    """
    Classe de usuário no banco de dados
    """

    __tablename__ = "users"

    telegram_id: Mapped[int] = mapped_column(BIGINT, primary_key=True)
    full_name: Mapped[str] = mapped_column(VARCHAR(255))
    username: Mapped[Optional[str]] = mapped_column(VARCHAR(255), nullable=True)
    language_code: Mapped[str] = mapped_column(VARCHAR(255))
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    referrer_id: Mapped[Optional[int]] = mapped_column(BIGINT, ForeignKey('users.telegram_id', ondelete='SET NULL'))
    orders: Mapped[list['Order']] = relationship(back_populates='user')
    
class Order(Base, TimestampMixin):
    """
    Tabela de pedidos
    """

    __tablename__ = "orders"

    order_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    user_id: Mapped[int] = mapped_column(BIGINT, ForeignKey("users.telegram_id", ondelete="CASCADE"))
    products: Mapped[list['OrderProducts']] = relationship()
    user: Mapped['User'] = relationship(back_populates='orders')
    
class Product(Base, TimestampMixin):
    """
    Tabela de produtos
    """
    
    __tablename__ = "products"

    product_id: Mapped[int] = mapped_column(INTEGER, primary_key=True)
    title: Mapped[str] = mapped_column(String(255))
    description: Mapped[str]
    price: Mapped[float] = mapped_column(DECIMAL(precision=16, scale=4))

class OrderProducts(Base):
    """
    Cria a tabela estrangeira do relacionamento entre pedido e produtos
    """
    
    __tablename__ = "order-products"

    order_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("orders.order_id", ondelete="CASCADE"), primary_key=True)
    product_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("products.product_id", ondelete="RESTRICT"), primary_key=True)
    quantity: Mapped[int]
    product: Mapped['Product'] = relationship()
    
# Você pode deletar todas as tabelas com esse comando
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.drop_all)

# E recria-las com esse comando
async with engine.begin() as conn:
    await conn.run_sync(Base.metadata.create_all)
    
await engine.dispose()

In [4]:
from faker import Faker
fake = Faker()

***
### Realizando um CRUD básico
***

In [5]:
from sqlalchemy import insert, select, delete


class Repo:
    """
    Repositorio
    """

    def __init__(self, session: AsyncSession):
        """
        Construtor
        """

        self.session = session
        
    async def add_user(
        self,
        telegram_id: int,
        full_name: str,
        language_code: str,
        username: str = None,
        referrer_id: int = None):
        """
        Adicionando usuários
        """

        stmt = insert(User).values(
            telegram_id=telegram_id,
            full_name=full_name,
            username=username,
            language_code=language_code,
            referrer_id=referrer_id,
        ).returning(User)
        result = await self.session.execute(stmt)
        await self.session.commit()
        return result.scalars().first()

    async def get_all_users(self):
        """
        Listando usuários
        """

        stmt = select(User)
        result = await self.session.execute(stmt)
        return result.scalars().all()

    async def cleanup_users(self):
        """
        Deletando usuários.
        """

        stmt = delete(User)
        await self.session.execute(stmt)
        await self.session.commit()

In [6]:
from faker import Faker

fake = Faker()

async with Session() as session:
    repo = Repo(session)
    
    await repo.cleanup_users()
    
    for _ in range(10):
        await repo.add_user(
            telegram_id=fake.pyint(),
            full_name=fake.name(),
            language_code=fake.language_code(),
            username=fake.user_name(),
        )
        
    for user in await repo.get_all_users():
        print(user.full_name)


Phillip Stafford
Tiffany Jones
Whitney Miller
Antonio Wells
Stephanie Jones
Ronald Wilson
James Baker
Alexis Carter
John Butler
Stephen Vaughn


## Controle de pool de conexão para requisições async
***

Se o código que faz CRUD no banco de dados de forma async precisamos ter controle do pool de conexão do banco, com isso devemos utilizar semaforos para controlar a quantidade de itens que será executado de forma concorrente.

In [ ]:
import asyncio

In [ ]:
semaphore = asyncio.Semaphore(value=50)

In [ ]:
async def create_register(self, payload: dict):
    async with semaphore:
        async with DBSession() as session:
            ...

In [ ]:
tasks = []
for i in range(10000):
    tasks.append(create_register())

In [ ]:
await asyncio.gather(**tasks)